# Hyper-Parameter Tunning Using GridSearchCV

In [ ]:
# For Example: We are classifying the Iris dataset based on the sepal. petal width and length to predict what type of
# flower it is. First question is which model should I choose there are So many models to choose from.

# For Instance we choose the SVM model now we have to choose hyperparameters there are so many values to choose from.

# The process of choosing the Optimal Parameters is called Hyperparameter Tunning.

In [2]:
# Loading the Iris dataset

from sklearn import datasets
iris = datasets.load_iris()

In [4]:
# Checking what the data contain

dir(iris)

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [6]:
# Making the dataframe 

import pandas as pd

df = pd.DataFrame(iris['data'], columns=iris['feature_names'])

# Adding the flower coloumn
df['flower'] = iris['target']
df['flower'] = df['flower'].apply(lambda x: iris['target_names'][x])
df[47:52]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor


In [8]:
# Splitting the dataset into training and testing

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(iris['data'], iris['target'], test_size=0.3)

In [10]:
# Training and Testing the SVM Model

from sklearn.svm import SVC
model = SVC(kernel = 'rbf', C = 30, gamma = 'auto')

model.fit(x_train, y_train)
model.score(x_test, y_test)

0.9333333333333333

In [ ]:
# Above we use the train-test split method and every time we run the train-test split method the distribution
# in the x_train and x_test changes and it change the score of the model.
# We cannot rely on this method because the score is changing based on the samples.

# To deal with this problem we have to use the K-Folds cross validation 

In [11]:
# Using K-Folds Cross validation

from sklearn.model_selection import cross_val_score

In [12]:
# Using cross_val_score with (kernel = 'linear', C = 10, gamma = 'auto')

cross_val_score(SVC(kernel = 'linear', C = 10, gamma = 'auto'), iris['data'], iris['target'], cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [13]:
# Using cross_val_score with (kernel = 'rbf', C = 10, gamma = 'auto')

cross_val_score(SVC(kernel = 'rbf', C = 10, gamma = 'auto'), iris['data'], iris['target'], cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [14]:
# Using cross_val_score with (kernel = 'rbf', C = 20, gamma = 'auto')

cross_val_score(SVC(kernel = 'rbf', C = 20, gamma = 'auto'), iris['data'], iris['target'], cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

In [ ]:
# Above we use SVM model using different parameter combinations for each of the combinations we can get the scores
# we can take the average of these and find the average score and based on this we can find the optimal values
# for the parameters.

# But this method is very manual and repetitive there are so many values we can supply as a combinations of paramaters
# and everytime we have to wirte the code with different combinations. What is the solution of this?

# One approach is use the for loop.

In [15]:
# Using the for loop

import numpy as np

kernels = ['rbf', 'linear']
C = [1,10,20]
avg_scores = {}

for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(SVC(kernel=kval, C=cval, gamma='auto'), iris['data'], iris['target'], cv=5)
        avg_scores[kval+'_'+str(cval)] = np.average(cv_scores)
        
avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

In [ ]:
# By using the above apporach we can find the optimal scores using the Hyperparameter tunning.

# This approach also has some issues which is if we have four parameters then we have to run four loops and there are
# too many iterations.

In [16]:
# The solutions of all the above problems the sklearn provide API called GridSEarchCV which will do the exact same
# thing which we do above. (Hyperparameter tunning)

In [17]:
# Using GridSearchCv

from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(SVC(gamma='auto'),{
    'C' : [1,10,20],
    'kernel' : ['rbf', 'linear']
}, cv = 5, return_train_score=False)

clf.fit(iris['data'], iris['target'])
clf.cv_results_

# GridSearchCV(model, parameter grid)

{'mean_fit_time': array([0.00280781, 0.00119524, 0.00204773, 0.00153871, 0.00119686,
        0.00091052]),
 'std_fit_time': array([0.00075102, 0.00039659, 0.00043808, 0.00109836, 0.00039995,
        0.00137738]),
 'mean_score_time': array([0.00222363, 0.00059857, 0.00160675, 0.00099235, 0.00065169,
        0.00366392]),
 'std_score_time': array([8.66866788e-04, 4.88736259e-04, 4.87678741e-04, 1.01616672e-05,
        5.40770205e-04, 6.34053148e-03]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20

In [18]:
# The GridSearchCV results are not easy to view but sklearn provide the way we can download the results in the dataframe

df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002808,0.000751,0.002224,0.000867,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.001195,0.000397,0.000599,0.000489,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.002048,0.000438,0.001607,0.000488,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.001539,0.001098,0.000992,0.000010,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.001197,0.000400,0.000652,0.000541,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.000911,0.001377,0.003664,0.006341,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [19]:
# Printing only relevant features

df[['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [20]:
# checking the properties of GridSearchCV classifier

dir(clf)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_doc_link_module',
 '_doc_link_template',
 '_doc_link_url_param_generator',
 '_estimator_type',
 '_format_results',
 '_get_default_requests',
 '_get_doc_link',
 '_get_metadata_request',
 '_get_param_names',
 '_get_routed_params_for_fit',
 '_get_scorers',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run

In [21]:
# checking the best score

clf.best_score_

0.9800000000000001

In [22]:
# checking the best parameters

clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [23]:
# The one issue with GridSearchCV is the computation cost there might be a dataset which consist of million of rows
# and also we have number of parameters.

# To tackle with the computation problem the sklearn comes up with RandomizedSearchCV
# RandomizedSearchCv will not try thr every combination and permutation of the parameters but it will try the random 
# combination of the parameter values

In [25]:
# Using RandomizedSearchCv

from sklearn.model_selection import RandomizedSearchCV

rs = RandomizedSearchCV(SVC(gamma='auto'), {
    'C' : [1,10,20],
    'kernel' : ['rbf', 'linear']
}, cv=5, return_train_score=False, n_iter=2)

rs.fit(iris['data'], iris['target'])
pd.DataFrame(rs.cv_results_)[['param_C', 'param_kernel', 'mean_test_score']]

# n_iter = 2  -> It defines the number of different sets of hyperparameters to try during the random search.
# cv = 5      -> This refers to the number of folds in cross-validation.

# Everytime we run it, it will change the parameters.

# This work  

,param_C,param_kernel,mean_test_score
0,20,rbf,0.966667
1,20,linear,0.966667


## Choosing the Best Model and Best Parameters

In [27]:
# Importing the Models

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [29]:
# Defining the Parameter Grid

model_params = {
    'svm' : {
        'model' : svm.SVC(gamma='auto'),
        'params' : {
            'C' : [1,10,20],
            'kernel' : ['rbf', 'linear']
        }
    },
    'random_forest' : {
        'model' : RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model' : LogisticRegression(solver='liblinear', multi_class='auto'),
        'params' : {
            'C' : [1,5,10]
        }
    }
}

In [38]:
# Using for loop

scores = []
 
for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris['data'], iris['target'])
    scores.append({
        'model' : model_name,
        'best_score' : clf.best_score_ ,
        'best_params' : clf.best_params_
    })

In [39]:
df = pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.960000,{'n_estimators': 5}
2,logistic_regression,0.966667,{'C': 5}
